In [9]:
!pip install pyautogui keyboard


In [11]:
import pyautogui
import time
import keyboard  # Ensure you install this with pip install keyboard

# Function to move the mouse to a specific location (x, y)
def move_mouse(x, y):
    pyautogui.moveTo(x, y)
    print(f"Moved mouse to ({x}, {y})")

# Function to click the mouse at the current position
def click_mouse():
    pyautogui.click()
    print("Mouse clicked")

# Function to right-click the mouse at the current position
def right_click_mouse():
    pyautogui.rightClick()
    print("Right mouse clicked")

# Function to scroll the mouse up or down
def scroll_mouse(amount):
    pyautogui.scroll(amount)
    print(f"Scrolled mouse by {amount}")

# Function to drag the mouse from a start to an end position
def drag_mouse(start_x, start_y, end_x, end_y):
    pyautogui.moveTo(start_x, start_y)
    pyautogui.mouseDown()  # Hold mouse down to drag
    pyautogui.moveTo(end_x, end_y)  # Move to new location
    pyautogui.mouseUp()  # Release mouse button
    print(f"Dragged mouse from ({start_x}, {start_y}) to ({end_x}, {end_y})")

# Control function for interactive use of the virtual mouse
def control_mouse():
    current_x, current_y = pyautogui.position()  # Get current mouse position

    while True:
        # Move the mouse based on arrow keys
        if keyboard.is_pressed('up'):
            current_y -= 10
            move_mouse(current_x, current_y)
        elif keyboard.is_pressed('down'):
            current_y += 10
            move_mouse(current_x, current_y)
        elif keyboard.is_pressed('left'):
            current_x -= 10
            move_mouse(current_x, current_y)
        elif keyboard.is_pressed('right'):
            current_x += 10
            move_mouse(current_x, current_y)

        # Click mouse when space is pressed
        elif keyboard.is_pressed('space'):
            click_mouse()

        # Right-click mouse when 'r' is pressed
        elif keyboard.is_pressed('r'):
            right_click_mouse()

        # Drag mouse when 'd' is pressed
        elif keyboard.is_pressed('d'):
            drag_mouse(current_x, current_y, current_x + 100, current_y + 100)

        # Scroll mouse up or down based on key presses
        elif keyboard.is_pressed('w'):  # 'w' for scroll up
            scroll_mouse(200)
        elif keyboard.is_pressed('s'):  # 's' for scroll down
            scroll_mouse(-200)

        # Exit the program when 'esc' is pressed
        elif keyboard.is_pressed('esc'):
            print("Exiting program...")
            break

        time.sleep(0.1)

# Main program entry point
if __name__ == "__main__":
    print("Use arrow keys to move the mouse, space to click, 'r' to right-click, 'd' to drag, 'w' to scroll up, 's' to scroll down, and Esc to exit.")
    control_mouse()


Use arrow keys to move the mouse, space to click, 'r' to right-click, 'd' to drag, 'w' to scroll up, 's' to scroll down, and Esc to exit.
Exiting program...


In [13]:
!pip install opencv-python mediapipe pyautogui numpy


In [2]:
import pyautogui

# Disable PyAutoGUI's fail-safe feature
pyautogui.FAILSAFE = False

In [4]:
import cv2
import mediapipe as mp
import pyautogui
import time
import math

# Disable PyAutoGUI's fail-safe feature
pyautogui.FAILSAFE = False

# Initialize MediaPipe Hand model and pyautogui for mouse control
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Screen size for mouse control
screen_width, screen_height = pyautogui.size()

# Variables to track previous state of the hands
prev_x, prev_y = 0, 0
drag_start_time = 0
dragging = False

# Function to calculate distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Function to move the mouse based on hand position
def move_mouse(x, y):
    # Normalize the position to screen size
    x = int(screen_width * x)
    y = int(screen_height * y)
    pyautogui.moveTo(x, y)

# Function to click the mouse
def click_mouse(button='left'):
    if button == 'left':
        pyautogui.click(button='left')
    elif button == 'right':
        pyautogui.click(button='right')

# Function to scroll the mouse
def scroll_mouse(amount):
    pyautogui.scroll(amount)

# Function to drag the mouse
def drag_mouse(x1, y1, x2, y2):
    pyautogui.moveTo(x1, y1)
    pyautogui.mouseDown()
    pyautogui.moveTo(x2, y2)
    pyautogui.mouseUp()q

# Start capturing video
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a later mirrored display
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB (for MediaPipe)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    # Draw hand landmarks and detect gestures
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get the coordinates of index and thumb fingertips (important for gestures)
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

            # Normalize the coordinates to screen size
            x_index = index_finger_tip.x
            y_index = index_finger_tip.y
            x_thumb = thumb_tip.x
            y_thumb = thumb_tip.y

            # Mouse movement based on the index finger position
            move_mouse(x_index, y_index)

            # Distance between index and thumb for click detection
            distance = calculate_distance(x_index, y_index, x_thumb, y_thumb)

            # Left click (pinch gesture)
            if distance < 0.05:  # If thumb and index are close enough
                click_mouse('left')

            # Right click (pinch gesture with other hand)
            if distance < 0.05 and index_finger_tip.x < 0.5:  # Adjust based on specific hand position
                click_mouse('right')

            # Drag (if the index and thumb are held for more than 1 second)
            if distance < 0.05 and not dragging:
                drag_start_time = time.time()
                dragging = True
            if dragging and (time.time() - drag_start_time > 1):
                drag_mouse(prev_x, prev_y, x_index, y_index)
                dragging = False

            # Scroll action (when middle finger is above/below the index finger)
            if middle_finger_tip.y < index_finger_tip.y:
                scroll_mouse(50)  # Scroll up
            elif middle_finger_tip.y > index_finger_tip.y:
                scroll_mouse(-50)  # Scroll down

            # Update previous mouse positionq
            prev_x, prev_y = x_index, y_index

    # Display the resulting frame
    cv2.imshow("Virtual Mouse", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


C:\ProgramData\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
